In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

import plotly
plotly.offline.init_notebook_mode(connected=True) 

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#That's a ReMix that I copied from the last Feedback Prize - Evaluating Student Writing

https://www.kaggle.com/competitions/feedback-prize-2021/overview

In [ ]:
#Code by Dino Wun https://www.kaggle.com/code/dinowun/eda-simplified-feedback-prize

# ℹ️: You can copy file path in data sections if you would want!
TRAIN_DIR = "../input/feedback-prize-effectiveness/train"
TEST_DIR = "../input/feedback-prize-effectiveness/test"
train_files = os.listdir(TRAIN_DIR)
test_files = os.listdir(TEST_DIR)

for file in range(len(train_files)):
    train_files[file] = TRAIN_DIR + "/" + str(train_files[file])
    
for file in range(len(test_files)):
    test_files[file] = TEST_DIR + "/" + str(test_files[file])
    
train_df = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
sub_df = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")

#Distribution of Discourse Type

In [ ]:
#Code by Dino Wun https://www.kaggle.com/code/dinowun/eda-simplified-feedback-prize

fig = px.bar(x=np.unique(train_df["discourse_type"]), y=[list(train_df["discourse_type"]).count(i) for i in np.unique(train_df["discourse_type"])], color=np.unique(train_df["discourse_type"]), color_continuous_scale="Mint")
fig.update_xaxes(title="Classes")
fig.update_yaxes(title="Number of Rows")
fig.update_layout(showlegend=True, 
                  title={
                      'text':'Discourse Type Distribution', 
                      'y':0.95, 
                      'x':0.5, 
                      'xanchor':'center', 
                      'yanchor':'top'}, template="seaborn")
fig.show()

#Distribution of Discourse Effectiveness

In [ ]:
#Code by Dino Wun https://www.kaggle.com/code/dinowun/eda-simplified-feedback-prize

fig = px.bar(x=np.unique(train_df["discourse_effectiveness"]), y=[list(train_df["discourse_effectiveness"]).count(i) for i in np.unique(train_df["discourse_effectiveness"])], color=np.unique(train_df["discourse_effectiveness"]), color_continuous_scale="Mint")
fig.update_xaxes(title="Classes")
fig.update_yaxes(title="Number of Rows")
fig.update_layout(showlegend=True, 
                  title={
                      'text':'Enumerated class label of Discourse Effectiveness Distribution', 
                      'y':0.95, 
                      'x':0.5, 
                      'xanchor':'center', 
                      'yanchor':'top'}, template="seaborn")
fig.show()

#Let's wordcloud the words!

In [ ]:
#Code by Dino Wun https://www.kaggle.com/code/dinowun/eda-simplified-feedback-prize

import wordcloud

wordcloud = wordcloud.WordCloud(stopwords=wordcloud.STOPWORDS, max_font_size=90, max_words=4500, width=600, height=400, background_color='black', colormap='Set3').generate(' '.join(txt for txt in train_df["discourse_text"]))
fig, ax = plt.subplots(figsize=(14,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
plt.imshow(wordcloud);

In [ ]:
train_df = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
train_df.head()

#Distribution of Labels

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

train_df["discourse_type"].value_counts().plot(kind="bar");

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

vocab_size = 10000 # Vocabulary size
sequence_length = 1024 # Sequence Length
batch_size = 128 # Batch size
unk_token = "<UNK>" # Unknownd token
padding_token = "<PAD>"
none_class = "O"
vectorizer_path = "vectorizer.json"
# Use output dataset for inference
output_dataset_path = "../input/name-entity-with-transformer-output/"
model_path = "model.h5"
embed_size = 128
hidden_size = 64
modes = ["training", "inference"] # There is training and inference mode
mode = modes[0]
epochs = 30

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

discourse_types = list(train_df["discourse_type"].value_counts().index)

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

all_tags = [padding_token]
for discourse_type in discourse_types:
    all_tags.append("B-" + discourse_type)
for discourse_type in discourse_types:
    all_tags.append("I-" + discourse_type)
all_tags.append(none_class)
pad_index = all_tags.index(padding_token)
none_index = all_tags.index(none_class)
tag_index = dict([(tag, index) for (index, tag) in enumerate(all_tags)])
index_tag = dict([(tag_index[tag], tag.replace("B-", "").replace("I-", "")) for tag in tag_index])
print(all_tags)
print(tag_index)
print(index_tag)

#Tokenization

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

def tokenize(content):
    tokens = content.lower().split()
    return tokens
    
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output
def get_range(item):
    locations = [int(location) for location in item["predictionstring"].split(" ")]
    return (locations[0], locations[-1])
def add_annotation(all_data, start_index, end_index, discourse_type):
    for j in range(start_index, end_index): 
        if j == start_index:
            all_data[-1][1][j] = tag_index["B-" + discourse_type]
        else:
            all_data[-1][1][j] = tag_index["I-" + discourse_type]

#Vectorization

In [ ]:
#Code by Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

class Vectorizer:
    
    def __init__(self, vocab_size = None, sequence_length = None, unk_token = "<unk>"):
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.unk_token = unk_token
        
    def fit_transform(self, sentences):
        word_counter = dict()
        for tokens in sentences:
            for token in tokens: 
                if token in word_counter:
                    word_counter[token] += 1
                else:
                    word_counter[token] = 1
        word_counter = pd.DataFrame({"key": word_counter.keys(), "count": word_counter.values()})
        word_counter.sort_values(by="count", ascending=False, inplace=True)
        vocab = set(word_counter["key"][0:self.vocab_size-2])
        word_index = dict()
        begin_index = 1 
        word_index[self.unk_token] = begin_index
        begin_index += 1
        Xs = []
        for i in range(len(sentences)):
            X = []
            for token in sentences[i]:
                if token not in word_index and token in vocab:
                    word_index[token] = begin_index
                    begin_index += 1
                if token in word_index:
                    X.append(word_index[token])
                else:
                    X.append(word_index[self.unk_token])
                if len(X) == self.sequence_length:
                    break
            Xs.append(X)
        self.word_index = word_index
        self.vocab = vocab
        return Xs
    
    def transform(self, sentences):
        Xs = []
        for i in range(len(sentences)):
            X = []
            for token in sentences[i]:
                if token in self.word_index:
                    X.append(self.word_index[token])
                else:
                    X.append(self.word_index[self.unk_token])
                if len(X) == self.sequence_length:
                    break
            Xs.append(X)
        return Xs
    
    def load(self, path):
        with open(path, 'r') as f:
            dic = json.load(f)
            self.vocab_size = dic['vocab_size']
            self.sequence_length = dic['sequence_length']
            self.unk_token = dic['unk_token']
            self.word_index = dic['word_index']
            
    def save(self, path):
        with open(path, 'w') as f:
            data = json.dumps({
                "vocab_size": self.vocab_size, 
                "sequence_length": self.sequence_length, 
                "unk_token": self.unk_token,
                "word_index": self.word_index
            })
            f.write(data)

#Let's go to plan B since the code above requires another Dataset. 

#Preprocessing

In [ ]:
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

def clean_text(text, remove_stopwords=True, stem_words=False, lemma=True):
    text = str(text).lower().split()
    
    if remove_stopwords:
        stops = stopwords.words("english")
        text = [x for x in text if not x in stops]
        
    text = ' '.join(text)
    
    text = re.sub(r"[-()\"#/<>!@&;*:<>{}`'+=~%|.!?,_]", " ", text)
    text = re.sub(r"\]", " ", text)
    text = re.sub(r"\[", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"  ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"   ", " ", text)
    text = re.sub(r"0x00", "", text)
    
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stem_words = [stemmer.stem(x) for x in text]
        text = " ".join(text)
        
    if lemma:
        text = text.split()
        lem = WordNetLemmatizer()
        lemmatized = [lem.lemmatize(x, "v") for x in text]
        text = " ".join(text)
        
    return text

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

%time
train_df['cleaned_text'] = train_df.discourse_text.apply(clean_text)

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

train_df.iloc[0]['discourse_text']

#Nasa Again! :)

#TFIDF Vectorizer

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1000)
tfidf_vect.fit(train_df['cleaned_text'])

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

TEST_PATH = "../input/feedback-prize-effectiveness/test"

def get_test_text(a_id):
    a_file = f"{TEST_PATH}/{a_id}.txt"
    with open(a_file, "r") as fp:
        txt = fp.read()
    return txt

def create_df_test():
    test_ids = [f[:-4] for f in os.listdir(TEST_PATH)] #Remove the last 4 characters ('.txt') in the filenames such as '0FB0700DAF44.txt'.
    test_data = []
    for test_id in test_ids:
        text = get_test_text(test_id)
        sentences = nltk.sent_tokenize(text)
        id_sentences = []
        idx = 0 
        for sentence in sentences:
            id_sentence = []
            words = sentence.split()
            # I created this heuristic for mapping words in sentences to "word indices"
            # This is not definitive and might have strong drawbacks and problems
            for w in words:
                id_sentence.append(idx)
                idx+=1
            id_sentences.append(id_sentence)
        test_data += list(zip([test_id] * len(sentences), sentences, id_sentences))
    df_test = pd.DataFrame(test_data, columns=['id', 'discourse_text', 'ids'])
    return df_test

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

df_test = create_df_test()
df_test.head()

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

df = train_df.append(df_test)

#Handling with Nan to avoid an error below on test_tfidf

In [ ]:
# categorical features with missing values
categorical_nan = [feature for feature in df.columns if df[feature].isna().sum()>0 and df[feature].dtypes=='O']
print(categorical_nan)

In [ ]:
# replacing missing values in categorical features
for feature in categorical_nan:
    df[feature] = df[feature].fillna('None')

In [ ]:
df[categorical_nan].isna().sum()

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

train_tfidf = tfidf_vect.fit_transform(df[:train_df.shape[1]]['cleaned_text'])
test_tfidf = tfidf_vect.transform(df[train_df.shape[1]:]['cleaned_text'])#Original is [0]

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

features = tfidf_vect.get_feature_names()
features[:20]

#Random Forest

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

rf_model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
#scores = cross_val_score(rf_model, train_tfidf, train_df['discourse_type'], scoring='accuracy', cv=cv)

In [ ]:
#Code by Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier

#%%time
#rf_model.fit(train_tfidf, train_df['discourse_type'])

#After the error above I gave up

ValueError: Found input variables with inconsistent numbers of samples: [6, 36765]

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

import pandas as pd, os
import cudf, cuml, cupy
from tqdm import tqdm
import numpy as np
print('RAPIDS',cudf.__version__)

In [ ]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
train_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-effectiveness/train'))):
    train_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-effectiveness/train/' + f, 'r').read())
train_text_df = cudf.DataFrame({'id': train_names, 'text': train_texts})
train_text_df.head()

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

from cuml.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', binary=True, max_features=25_000)
text_embeddings = tfidf.fit_transform( train_text_df.text ).toarray()

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

from cuml import UMAP
umap = UMAP()
embed_2d = umap.fit_transform(text_embeddings)
embed_2d = cupy.asnumpy( embed_2d )

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

from cuml import KMeans
kmeans = cuml.KMeans(n_clusters=15)
kmeans.fit(embed_2d)
train_text_df['cluster'] = kmeans.labels_

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

import matplotlib.pyplot as plt

centers = kmeans.cluster_centers_

plt.figure(figsize=(10,10))
plt.scatter(embed_2d[:,0], embed_2d[:,1], s=1, c=kmeans.labels_)
plt.title('UMAP Plot of Train Text using Tfidf features\nRAPIDS Discovers the 15 essay topics!',size=16)

for k in range(len(centers)):
    mm = cupy.mean( text_embeddings[train_text_df.cluster.values==k],axis=0 )
    ii = cupy.argmax(mm)
    top_word = tfidf.vocabulary_.iloc[ii]
    plt.text(centers[k,0]-1,centers[k,1]+0.75,f'{k+1}-{top_word}',size=16)

plt.show()

In [ ]:
#Code by Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics

for k in range(15):
    mm = cupy.mean( text_embeddings[train_text_df.cluster.values==k],axis=0 )
    ii = cupy.asnumpy( cupy.argsort(mm)[-5:][::-1] )
    top_words = tfidf.vocabulary_.to_array()[ii]
    print('#'*25)
    print(f'### Essay Topic {k+1}')
    print('### Top 5 Words',top_words)
    print('#'*25)
    tmp = train_text_df.loc[train_text_df.cluster==k].sample(3, random_state=123)
    for j in range(3):
        txt = tmp.iloc[j,1]
        print('-'*10,f'Example {j+1}','-'*10)
        print(txt,'\n')

#That's all from Feedback Prize Effectiveness ReMix

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_jATyVNm5La9Ppm6jN0oDTw8_ppt8x8BfnQ&usqp=CAU)pinterest.com

#Acknowledgements:

Chris Deotte https://www.kaggle.com/code/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics  (Finally I could copy one of his brilliant codes)

Raghavendrakuttala https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533

Dino Wun https://www.kaggle.com/code/dinowun/eda-simplified-feedback-prize

Lonnie https://www.kaggle.com/code/lonnieqin/name-entity-recognition-with-transformer

Neha Pawar  https://www.kaggle.com/code/nehapawar/tfidf-random-forest-classifier